In [7]:
import re
import pandas as pd
import csv

class Appearance:
    """
    Represents the appearance of a term in a given document, along with the
    frequency of appearances in the same one.
    """
    def __init__(self, docId, frequency):
        self.docId = docId
        self.frequency = frequency
        
    def __repr__(self):
        """
        String representation of the Appearance object
        """
        return str(self.__dict__)

In [8]:
class Database:
    """
    In memory database representing the already indexed documents.
    """
    def __init__(self):
        self.db = dict()

    def __repr__(self):
        """
        String representation of the Database object
        """
        return str(self.__dict__)
    
    def get(self, id):
        return self.db.get(id, None)
    
    def add(self, document):
        """
        Adds a document to the DB.
        """
        return self.db.update({document['id']: document})

    def remove(self, document):
        """
        Removes document from DB.
        """
        return self.db.pop(document['id'], None)

In [9]:
class InvertedIndex:
    """
    Inverted Index class.
    """
    def __init__(self, db):
        self.index = dict()
        self.db = db
    
    def returnIndex(self):
        return self.index
    
    def exportDictToCSV(self):
        with open('mycsvfile.csv','wb') as f:
            w = csv.writer(f)
            w.writerows(self.index.items())

    def __repr__(self):
        """
        String representation of the Database object
        """
        return str(self.index)
        
    def index_document(self, document):
        """
        Process a given document, save it to the DB and update the index.
        """
        
        # Remove punctuation from the text.
        clean_text = re.sub(r'[^\w\s]','', document['text'])
        terms = clean_text.split(' ')
        appearances_dict = dict()
        # Dictionary with each term and the frequency it appears in the text.
        for term in terms:
            term_frequency = appearances_dict[term].frequency if term in appearances_dict else 0
            appearances_dict[term] = Appearance(document['id'], term_frequency + 1)
            
        # Update the inverted index
        update_dict = { key: [appearance]
                       if key not in self.index
                       else self.index[key] + [appearance]
                       for (key, appearance) in appearances_dict.items() }
        self.index.update(update_dict)
        # Add the document into the database
        self.db.add(document)
        return document
    
    def lookup_query(self, query):
        """
        Returns the dictionary of terms with their correspondent Appearances. 
        This is a very naive search since it will just split the terms and show
        the documents where they appear.
        """
        return { term: self.index[term] for term in query.split(' ') if term in self.index }

In [18]:
import pandas as pd
data = pd.read_csv('../Data/ProcessedTweets.csv');
#data.columns = ['id','content']
#data = data[:]

In [19]:
def highlight_term(id, term, text):
    replaced_text = text.replace(term, "\033[1;32;40m {term} \033[0;0m".format(term=term))
    return "--- document {id}: {replaced}".format(id=id, replaced=replaced_text)

db = Database()
index = InvertedIndex(db)

for i in range(len(data)):
    text = data['content'][i]
    if(pd.isnull(text)):
        continue
    document = {
        'id': data['id'][i],
        'text': data['content'][i]
    }
    index.index_document(document)


In [20]:
import json
search_term = "colombia"
result = index.lookup_query(search_term)
print(result)
dic = index.returnIndex().__repr__
print(len(index.returnIndex().keys()))
with open('index.json', 'w') as json_file:  
    json.dump(dic(), json_file)
for term in result.keys():
    for appearance in result[term]:
        # Belgium: { docId: 1, frequency: 1}
        document = db.get(appearance.docId)
        print(highlight_term(appearance.docId, term, document['text']))
        print(data.iloc[appearance.docId]['id_str'])
        print()
    print("-----------------------------")    

{'colombia': [{'docId': 5110, 'frequency': 1}, {'docId': 15032, 'frequency': 1}, {'docId': 21276, 'frequency': 1}]}
33931
--- document 5110: great honor welcome president manuel santos  colombia  white house today!????????joint press 
8.653375132459131e+17

--- document 15032: stickmissosolog realdonaldtrump universe  colombia  
5.5844327756610765e+17

--- document 21276: believe model mysteriously disappear obamacare website citizen  colombia  
4.00639951831724e+17

-----------------------------
